# 1D and 2D density plots
These scripts make use of Gromacs analysis tool data (xvg files). First, we set some general parameters that define what the plots look like. These you could copy from previous notebooks to achieve uniform style.

In [ ]:
import numpy as np
import scipy

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
#%config InlineBackend.figure_format = 'pdf'
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 300

plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

# The following require a local LaTex installation. If available, uncomment.

#plt.rcParams['text.usetex'] = True
#plt.rcParams['font.family'] = "sans-serif"
#plt.rcParams['font.serif'] = "cm"
#plt.rcParams['text.latex.preamble'] = r"\usepackage{subdepth}, \usepackage{type1cm}"

# Loading the data

The data is from a continuous trajectory divided into 5 pieces.
It contains the center of mass coordinates of each cholesterol molecule in the trajectory

In [ ]:
N=5
coor={}
for i in range(0,N):
    coor[i]=np.loadtxt('chol_com_coord_' + str(i) + '.xvg')

# Calculating the densities of cholesterol along the membrane normal

Extract z y and z coordiantes from the loaded data and calculate the distribution along z.


In [ ]:
binedges=np.arange(7,14,0.05)
for i in range(0,N): 
    # every third column is x starting from column 1
    x=coor[i][:,1::3].reshape(-1)
    # every third column is y starting from column 1
    y=coor[i][:,2::3].reshape(-1)
    # every third column is z starting from column 1
    z=coor[i][:,3::3].reshape(-1)
    hist , binedges = np.histogram(z, bins=binedges, density=True)
    bincenters=binedges[1:]/2.+binedges[0:-1]/2.
    plt.plot(bincenters, hist)
    plt.xlabel(r'$z$ / nm')
    plt.ylabel(r'$n$')

    mask= (bincenters>10) * (bincenters<12)

# Calculating the 2D densities

1) Generate the grid

In [ ]:
dx=0.2
dy=0.2
xedges=np.arange(-4,12,dx)
yedges=np.arange(-3,13,dy)

2) Make the histogram seperately for each data file

In [ ]:
H_a={}
for i in range(0,N):   
    x=coor[i][:,1::3].reshape(-1)
    y=coor[i][:,2::3].reshape(-1)
    z=coor[i][:,3::3].reshape(-1)
    H, _xedges,_yedges = np.histogram2d(x,y, bins=(xedges,yedges), density=True)
    H_a[i]=H

3) Histogram and plot the data

In [ ]:
xc=(xedges[1:]+xedges[:-1]) / 2.0
yc=(yedges[1:]+yedges[:-1]) / 2.0

for i in range(0,N):     
    H=H_a[i].copy()
    H[H==0]=np.nan
    plt.matshow(H, cmap=plt.cm.gray_r, interpolation='gaussian', vmin=0, vmax=0.08)
    plt.xlabel(r'$x$ / nm')
    plt.ylabel(r'$y$ / nm')
    ax = plt.gca();
    ax.set_xticks(np.arange(0, H.shape[0],20))
    ax.set_yticks(np.arange(0, H.shape[0],20))
    ax.set_xticklabels(xc[::20].astype(int))
    ax.set_yticklabels(yc[::20].astype(int))
    cb=plt.colorbar()
    cb.set_label(r'$\rho$ / nm$^{-1}$')

4) The above density is for both leaflets. Below, I seperate the leaflets of the membrane and plot choelsterol distribution seperately.

Upper leaflet

In [ ]:
H_u={}
for i in range(0,N):   
    z=coor[i][:,3::3].reshape(-1)
    # I determined that z greater thatn 11 s the upper leaflet
    leaflet=z>11
    x=coor[i][:,1::3].reshape(-1)[leaflet]
    y=coor[i][:,2::3].reshape(-1)[leaflet]
    
    H, _xedges,_yedges = np.histogram2d(x,y, bins=(xedges,yedges), density=True)
    H_u[i]=H

In [ ]:
xc=(xedges[1:]+xedges[:-1]) / 2.0
yc=(yedges[1:]+yedges[:-1]) / 2.0

for i in range(0,N):     
    H=H_u[i].copy()
    H[H==0]=np.nan
    plt.matshow(H, cmap=plt.cm.gray_r, interpolation='gaussian', vmin=0, vmax=0.15)
    plt.xlabel(r'$x$ / nm')
    plt.ylabel(r'$y$ / nm')
    ax = plt.gca();
    ax.set_xticks(np.arange(0, H.shape[0],20))
    ax.set_yticks(np.arange(0, H.shape[0],20))
    ax.set_xticklabels(xc[::20].astype(int))
    ax.set_yticklabels(yc[::20].astype(int))
    cb=plt.colorbar()
    cb.set_label(r'$\rho$ / nm$^{-1}$')

Lower Leaflet

In [ ]:
H_l={}
for i in range(0,N):   
    z=coor[i][:,3::3].reshape(-1)
    # I determined that z greater thatn 11 s the upper leaflet
    leaflet=z<=11
    x=coor[i][:,1::3].reshape(-1)[leaflet]
    y=coor[i][:,2::3].reshape(-1)[leaflet]
    
    H, _xedges,_yedges = np.histogram2d(x,y, bins=(xedges,yedges), density=True)
    H_l[i]=H

In [ ]:
xc=(xedges[1:]+xedges[:-1]) / 2.0
yc=(yedges[1:]+yedges[:-1]) / 2.0

for i in range(0,N):     
    H=H_l[i].copy()
    H[H==0]=np.nan
    plt.matshow(H, cmap=plt.cm.gray_r, interpolation='gaussian', vmin=0, vmax=0.15)
    plt.xlabel(r'$x$ / nm')
    plt.ylabel(r'$y$ / nm')
    ax = plt.gca();
    ax.set_xticks(np.arange(0, H.shape[0],20))
    ax.set_yticks(np.arange(0, H.shape[0],20))
    ax.set_xticklabels(xc[::20].astype(int))
    ax.set_yticklabels(yc[::20].astype(int))
    cb=plt.colorbar()
    cb.set_label(r'$\rho$ / nm$^{-1}$')

# Block bootstrap for a simple implementation to estimate errors.

I could use the standard error estimate, but I wanted to try bootstrapping.
I have the data in pieces, I consider each piece an independent block. 

In [ ]:
Nboot=100

mu_a=np.zeros(H_a[0].shape)
std_a=np.zeros(H_a[0].shape)

mu_u=np.zeros(H_u[0].shape)
std_u=np.zeros(H_u[0].shape)

mu_l=np.zeros(H_l[0].shape)
std_l=np.zeros(H_l[0].shape)


for bs in range(0,Nboot):
    # Generate a list of 5 random numbers that determines the roder of drawing blocks
    blocks=np.random.randint(1,N,size=N-1)
    
    # reestimate the density based on bootstrapped data
    H=sum([H_a[i] for i in blocks])/blocks.size
    # acumulate to calculate the mean and the standard deviation later
    mu_a+=H
    std_a+=H**2.0
    
    # do the same for the upper and the lower leaflets
    H=sum([H_u[i] for i in blocks])/blocks.size
    mu_u+=H
    std_u+=H**2.0
    
    H=sum([H_l[i] for i in blocks])/blocks.size
    mu_l+=H
    std_l+=H**2.0

# Get the means and the standard deviations.  With bootstrap std = error    
mu_a/=Nboot
std_a=(std_a/Nboot-mu_a**2.0)**0.5    
 
mu_u/=Nboot
std_u=(std_u/Nboot-mu_u**2.0)**0.5
    
mu_l/=Nboot
std_l=(std_l/Nboot-mu_l**2.0)**0.5

Plot the average denstity and the 2D error

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

# for colorbar tick marks
from matplotlib import ticker

# for scale bar
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
#import matplotlib.font_manager as fm
#fontprops = fm.FontProperties(size=18)

dict_mu={}
dict_mu[0]=mu_a
dict_mu[1]=mu_u
dict_mu[2]=mu_l

dict_std={}
dict_std[0]=std_a
dict_std[1]=std_u
dict_std[2]=std_l

#print(mu_a[np.isfinite(mu_a)])

for i in range(0,3):
    mu=dict_mu[i].copy()
    mu[mu==0]=np.nan
    std=dict_std[i].copy()
    std[std==0]=np.nan 
    
    f, ax = plt.subplots(1, 2, sharey=False)
    plt.subplots_adjust(wspace=0.8)
    
    p0=ax[0].matshow(mu, cmap=plt.cm.gray_r, interpolation='none', vmin=0, vmax=0.05)
    p1=ax[1].matshow(std, cmap=plt.cm.gray_r, interpolation='none', vmin=0, vmax=0.02)


    ax_pos = ax[0].get_position()
    cb_pos = [(ax_pos.x0 + ax_pos.width)+ax_pos.width*0.05, ax_pos.y0,  ax_pos.width / 20.0, ax_pos.height]
    cax = f.add_axes(cb_pos)
    cb0 = plt.colorbar(p0,cax=cax)
   
    ax_pos = ax[1].get_position()
    cb_pos = [(ax_pos.x0 + ax_pos.width)+ax_pos.width*0.05, ax_pos.y0,  ax_pos.width / 20.0, ax_pos.height]
    cax = f.add_axes(cb_pos)
    cb1 = plt.colorbar(p1,cax=cax)
           
    for j in range(0,2):    
        #ax[j].set_xlabel(r'$x$ / nm')
        #ax[j].set_ylabel(r'$y$ / nm')
        ax[j].set_xticks([])
        ax[j].set_yticks([])
        ax[j].set_xticklabels(xc[::20].astype(int))
        ax[j].set_yticklabels(yc[::20].astype(int))
    
        scalebar = AnchoredSizeBar(ax[j].transData,
                           5, str(dx*5)+' nm', 'lower right', 
                           pad=0.1,
                           color='black',
                           frameon=False,
                           size_vertical=1,
                           )

        ax[j].add_artist(scalebar)
    
    cb0.set_label(r'$\rho$ / nm$^{-1}$')
    cb1.set_label(r'$\epsilon_\rho$ / nm$^{-1}$')

    #cb0.ax.set_yticklabels(cb0.ax.get_yticklabels(),rotation='vertical')
    tick_locator = ticker.MaxNLocator(nbins=2)
    cb0.locator = tick_locator
    cb0.update_ticks()
    
    tick_locator = ticker.MaxNLocator(nbins=2)
    cb1.locator = tick_locator
    cb1.update_ticks()